# Pizza pizza pizza

Get top venues by keyword and location, querying Google, Yelp, Foursquare

#### Motivation:
 - Gmaps and Yelp GUIs sorting and filtering are limited.
 - Query all 3 services and bring combined results into sortable table and map widget

#### Google

 - Needs a Google API key and module
 - [Create Google Cloud credentials and give access to Places APIs](https://console.cloud.google.com/google/maps-apis/credentials) (also, restrict IP or set other restrictions)
 - `conda install -c conda-forge -y gmaps`
 - put key in `apikey.txt`
 - `gmaps` Jupyter nbextension to show maps in notebook, with marker pins etc.

```
conda install -c conda-forge -y jupyter_contrib_nbextensions
jupyter nbextension enable --py gmaps
jupyter notebook
```

#### Yelp
 - needs Yelp API key and module
 - https://www.yelp.com/developers/documentation/v3
 - https://github.com/gfairchild/yelpapi
 - put key in `yelpkey.txt`
 
#### Foursquare
- Needs Foursquare API key and module
- https://developer.foursquare.com/docs/places-api/getting-started/
- https://github.com/mLewisLogic/foursquare
- OAuth id in `foursquare_id.txt`
- OAuth secret in `foursquare_secret.txt`

See `requirements.txt` for versions used, other requirements (requests, beakerx, folium, Flask)

Was going to try OpenTable and TripAdvisor but their language seems to limit API key access to approved commercial partners.
- https://dev.opentable.com/affiliate-partners/
- https://www.tripadvisor.com/APIAccessSupport

In [12]:
import time
from pprint import pprint
import ipywidgets
from ipywidgets import widgets, interact
from itertools import product
from os import path

import traceback
import pdb

import numpy as np
import pandas as pd

# cluster similar entities
import pandas_dedupe

import sklearn
from sklearn.preprocessing import StandardScaler

import requests, json 

import qgrid
from qgrid import show_grid

# convert coords to km using haversine distance 
import geopy
from geopy.distance import distance

import gmaps
with open('apikeys/apikey.txt') as f:
    api_key = f.readline().strip()
    f.close
gmaps.configure(api_key=api_key)

# https://github.com/gfairchild/yelpapi
import yelpapi
from yelpapi import YelpAPI
with open('apikeys/yelpkey.txt') as f:
    yelp_key = f.readline().strip()
    f.close
yelp_api = YelpAPI(yelp_key)

import foursquare
from foursquare import Foursquare, FoursquareException
with open('apikeys/foursquare_id.txt') as f:
    foursquare_id = f.readline().strip()
    f.close
with open('apikeys/foursquare_secret.txt') as f:
    foursquare_secret = f.readline().strip()
    f.close

# interactive maps
import folium

print("%-20s %s" % ("numpy", np.__version__))
print("%-20s %s" % ("pandas", pd.__version__))
print("%-20s %s" % ("ipywidgets", ipywidgets.__version__))
print("%-20s %s" % ("qgrid", qgrid.__version__))
print("%-20s %s" % ("sklearn", sklearn.__version__))
print("%-20s %s" % ("requests", requests.__version__))
print("%-20s %s" % ("geopy", geopy.__version__))
print("%-20s %s" % ("gmaps", gmaps.__version__))
print("%-20s %s" % ("foursquare", foursquare.__version__))
print("%-20s %s" % ("folium", folium.__version__))


numpy                1.18.1
pandas               1.0.3
ipywidgets           7.5.1
qgrid                1.3.1
sklearn              0.22.1
requests             2.23.0
geopy                2.0.0
gmaps                0.9.0
foursquare           1!2020.1.30
folium               0.11.0


## Google Maps

In [2]:
# pick a search term
keyword_options = [('Pizza', 'pizza'), ('Coffee', 'coffee')]
keyword = 'pizza'

@interact
def get_kw(kw = widgets.Dropdown(
    options=keyword_options,
    value=keyword,
    description='Search term:  ',
    disabled=False,
)):
    global keyword
    keyword = kw
    return None


interactive(children=(Dropdown(description='Search term:  ', options=(('Pizza', 'pizza'), ('Coffee', 'coffee')…

In [3]:
# pick a location
location = '40.7484, -73.9857'
location_coords = tuple(eval(location))
location_options = [('Midtown', '40.7484, -73.9857'),
                    ('Downtown', '40.7077443,-74.0139089'),
                    ('Upper East Side', '40.7711473,-73.9661166'),
                    ('Upper West Side', '40.778794,-73.984257'),
                    ('Brooklyn Heights', '40.6915812,-73.9954095'), 
                    ('Grand Army Plaza', '40.671872,-73.972544'),
                    ('Bay Ridge', '40.624468,-74.0487134'),
                    ('Williamsburg', '40.7144609,-73.9553373'),
                  ]

@interact
def get_loc(loc = widgets.Dropdown(
    options=location_options,
    value=location,
    description='Location:',
)):
    global location
    global location_coords
    location = loc
    location_coords = tuple(eval(loc))
    return None

interactive(children=(Dropdown(description='Location:', options=(('Midtown', '40.7484, -73.9857'), ('Downtown'…

In [4]:
# pick anything we don't have a pickle file for
for test_keyword, test_location in product(keyword_options, location_options):
    keyword = test_keyword[1]
    location_name = test_location[0]
    location = test_location[1]
    location_coords = tuple(eval(test_location[1]))
    test_pickle_filename = keyword + "_" + location_name.replace(' ', '').lower() + ".pkl"
    
    if not path.exists(test_pickle_filename):
        break
        
print(location_name, keyword, location, location_coords)
print(test_pickle_filename)

Williamsburg coffee 40.7144609,-73.9553373 (40.7144609, -73.9553373)
coffee_williamsburg.pkl


In [5]:
# gmap of chosen location
figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(center=location_coords, zoom_level=12, layout=figure_layout)
fig.add_layer(gmaps.marker_layer([location_coords]))
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [6]:
# global options for all search services APIs
MIN_USER_RATINGS = 40
MIN_RATING = 3.5
NRESULTS = 50
RADIUS = 1000


In [7]:
# gmaps options
GMAPS_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# https://developers.google.com/places/web-service/search#TextSearchRequests
#https://developers.google.com/places/web-service/supported_types
# rankby='prominence'
rankby='distance'
ltype='establishment'


In [8]:
def gmaps_get_first_page(api_key, location, **kwargs):
    """get first page of results from gmaps using api_key, location, kwargs for search spec"""
    # use either rankby or radius kwarg
    request_url = GMAPS_URL + '?key=' + api_key
    request_url += '&location=' + location
    for name, val in kwargs.items():
        request_url += '&' + name + '=' + str(val)
    r = requests.get(request_url)
    j = r.json()
    return j


def gmaps_get_next_page(api_key, next_page_token):
    """get next search engine results page page using search token, waiting until available"""
    r = requests.get(GMAPS_URL + '?pagetoken=' + next_page_token +
                        '&key=' + api_key)
    for i in range(10):
        j = r.json()
        if not j['results']: # wait for next page to be available
            time.sleep(5)
            continue
        else:
            return j


def gmaps_get_all_df(api_key, location, **kwargs):
    """return dataframe of all results using api_key, location, search kwargs"""
    # get first page
    j = gmaps_get_first_page(api_key, location, **kwargs)
    venues_df = pd.json_normalize(j['results'])

    # get pages while additional pages available
    while 'next_page_token' in j:
        next_page_token = j['next_page_token']
        time.sleep(5)
        j = gmaps_get_next_page(api_key, next_page_token)
        venues_df = venues_df.append(pd.json_normalize(j['results']))
        
    return venues_df


def gmaps_get_df(location_coords, keyword):

    # use either rankby or radius
    location = "%.7f,%.7f" % location_coords
    gmaps_df = gmaps_get_all_df(api_key, location, keyword=keyword, ltype=ltype, radius=RADIUS)
    if gmaps_df.empty:
        return None
    else:
        # gmaps_get_df(api_key, location, keyword=keyword, ltype=ltype, radius=RADIUS)
        gmaps_df = gmaps_df.loc[(gmaps_df['user_ratings_total'] >= MIN_USER_RATINGS) & (gmaps_df['rating'] >= MIN_RATING)] \
                           .sort_values(['rating', 'user_ratings_total'], ascending=False) \
                           .reset_index(drop=True)
        gmaps_df = gmaps_df[['name', 'vicinity', 'rating', 'user_ratings_total', 'geometry.location.lat', 'geometry.location.lng']]
        gmaps_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng']
        # drop trailing ", Brooklyn"
        gmaps_df['address'] = gmaps_df['address'].apply(lambda address: " ".join(address.split(',')[:-1]))
        gmaps_df['distance'] = gmaps_df.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
                                              axis=1)
        return gmaps_df


In [9]:
%%time
gmaps_df = gmaps_get_df(location_coords, keyword)
gmaps_df = gmaps_df.loc[(gmaps_df['nratings'] >= MIN_USER_RATINGS) & (gmaps_df['rating'] >= MIN_RATING)] \
        .sort_values(['rating', 'nratings'], ascending=False) \
        .reset_index(drop=True)
gmaps_df


CPU times: user 71.5 ms, sys: 9.38 ms, total: 80.9 ms
Wall time: 7.17 s


,name,address,rating,nratings,lat,lng,distance
0,Stumptown Coffee Roasters,212b Pacific St,4.7,190,40.688831,-73.992174,0.410008
1,D'Amico Coffee,309 Court St,4.7,130,40.683479,-73.995175,0.899972
2,Kaigo Coffee Room,139 Bridge Park Dr,4.7,81,40.693491,-74.000994,0.517465
3,About coffee,187 State St,4.7,52,40.690322,-73.991653,0.346982
4,Blue Bottle Coffee,85 Dean St,4.6,392,40.687351,-73.989750,0.670440
5,East One Coffee Roasters,384 Court St,4.5,348,40.681188,-73.996620,1.158717
6,The Little Sweet Café,77 Hoyt St,4.5,240,40.688093,-73.986608,0.838769
7,WHITE NOISE COFFEE CO.,57 Smith St,4.5,71,40.689698,-73.988224,0.642386
8,Devoción,276 Livingston St,4.4,377,40.688557,-73.983459,1.064517
9,Absolute Coffee,327 Atlantic Ave,4.4,268,40.688194,-73.987637,0.757055


In [13]:
# sort grid by clicking on header (can also click on filter button)
show_grid(gmaps_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [14]:
# plot on google map

markers = [(row.lat, row.lng) for row in gmaps_df.itertuples()]
marker_hover = ["%s: %s (%s)" % (row.name, row.rating, row.nratings) for row in gmaps_df.itertuples()]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Google rating</dt><dd>{rating}</dd>
<dt>Google reviews</dt><dd>{nratings}</dd>
</dl>
"""
marker_info = [info_box_template.format(**row) for i, row in gmaps_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=eval(location), zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [15]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
    
venues_map

## Yelp

In [16]:
def yelp_get_df(location_coords, keyword):
    lat, lng = location_coords
    response = yelp_api.search_query(categories=keyword, latitude=lat, longitude=lng, 
                                     radius=RADIUS, sort_by=rankby, limit=NRESULTS)

    yelp_df = pd.json_normalize(response['businesses'])
    if not yelp_df.empty:    
        yelp_df = yelp_df.loc[(yelp_df['review_count'] >= MIN_USER_RATINGS) & (yelp_df['rating'] >= MIN_RATING)] \
                         .sort_values(['rating', 'review_count'], ascending=False) \
                         .reset_index(drop=True)
        display_columns = ['name', 'location.address1', 'rating', 'review_count', 'coordinates.latitude', 'coordinates.longitude', 'url']
        yelp_df = yelp_df[display_columns]
        yelp_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng', 'url']
        yelp_df['distance'] = yelp_df.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
                                              axis=1)
        
        return yelp_df
    else:
        return None

yelp_df = yelp_get_df(location_coords, keyword)
yelp_df

,name,address,rating,nratings,lat,lng,url,distance
0,Blossom Ice Cream,196 Court St,4.5,215,40.687279,-73.993973,https://www.yelp.com/biz/blossom-ice-cream-bro...,0.492972
1,Absolute Coffee,327 Atlantic Ave,4.5,177,40.688270,-73.987569,https://www.yelp.com/biz/absolute-coffee-brook...,0.757914
2,Blue Bottle Coffee,85 Dean St,4.5,139,40.687360,-73.989770,https://www.yelp.com/biz/blue-bottle-coffee-br...,0.668568
3,The Little Sweet Cafe,77B Hoyt St,4.5,134,40.688142,-73.986732,https://www.yelp.com/biz/the-little-sweet-cafe...,0.826947
4,White Noise Coffee Co,57 Smith St,4.5,75,40.689624,-73.988249,https://www.yelp.com/biz/white-noise-coffee-co...,0.643099
5,Kaigo Coffee Room,139 Bridge Park Dr,4.5,54,40.693608,-74.001017,https://www.yelp.com/biz/kaigo-coffee-room-bro...,0.524691
6,Brooklyn Farmacy & Soda Fountain,513 Henry St,4.0,691,40.683980,-73.999230,https://www.yelp.com/biz/brooklyn-farmacy-and-...,0.903768
7,Hanco's,134 Smith St,4.0,684,40.687032,-73.990429,https://www.yelp.com/biz/hancos-brooklyn?adjus...,0.657604
8,Van Leeuwen Ice Cream,81 Bergen St,4.0,319,40.686688,-73.990059,https://www.yelp.com/biz/van-leeuwen-ice-cream...,0.707003
9,Tutt Cafe,47 Hicks St,4.0,227,40.700188,-73.993220,https://www.yelp.com/biz/tutt-cafe-brooklyn?ad...,0.973519


In [19]:
show_grid(yelp_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [17]:
markers = [(row.lat, row.lng) for row in yelp_df.itertuples()]
marker_hover = ["%s: %s (%s)" % (row.name, row.rating, row.nratings) for row in yelp_df.itertuples()]

info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Yelp rating</dt><dd>{rating}</dd>
<dt>Yelp reviews</dt><dd>{nratings}</dd>
</dl>
"""

marker_info = [info_box_template.format(**row) for i, row in yelp_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=location_coords, zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [18]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map

## Foursquare

In [20]:
def foursquare_get_df(location_coords, keyword):
    location_str = "%.7f,%.7f" % location_coords
    client = Foursquare(client_id=foursquare_id, 
                                   client_secret=foursquare_secret, 
                                   redirect_uri='http://streeteye.com/')
    response = client.venues.search(params={'query': keyword, 'll': "%s" % location_str, 
                                            'radius': RADIUS, 'limit': NRESULTS})

    foursquare_array = []

    for i, venue in pd.json_normalize(response['venues']).iterrows():
        venue_id = venue['id']
        # query detailed venue info from foursquare
        venue_name = venue['name']
        venue_address = venue['location.address']
        venue_url = venue['delivery.url']
        venue_lat = venue['location.lat']
        venue_lng = venue['location.lng']
        # default these to -1
        venue_rating = -1
        venue_nratings = -1
        try:
            # get rating, nratings with another API call for venue details
            venue_details = client.venues(venue_id)['venue']
            venue_rating = venue_details['rating']
            venue_nratings = venue_details['ratingSignals']
        except FoursquareException as e:
            print("Foursquare exception", type(e), str(e))
        except Exception as e:
            pass
            # sometimes no rating ... probably not popular enough
            # print(type(e), str(e))
            # print(traceback.format_exc())
            # print("No rating for %s" % venue_name)

        foursquare_array.append([venue_name, venue_address, venue_rating, venue_nratings, venue_lat, venue_lng, venue_url])
            
    foursquare_df = pd.DataFrame(foursquare_array)
    if len(foursquare_df) and len(foursquare_df.columns):
        foursquare_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng', 'url']
        foursquare_df = foursquare_df.loc[(foursquare_df['nratings'] >= MIN_USER_RATINGS) & (foursquare_df['rating'] >= MIN_RATING)] \
                                     .sort_values(['rating', 'nratings'], ascending=False) \
                                     .reset_index(drop=True)
        foursquare_df['distance'] = foursquare_df.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
                                                        axis=1)
        return foursquare_df
    else:
        return None

foursquare_df = foursquare_get_df(location_coords, keyword)
foursquare_df

,name,address,rating,nratings,lat,lng,url,distance
0,Stumptown Coffee Roasters,212 Pacific St,9.2,207,40.688942,-73.992141,NaN,0.402799
1,Blue Bottle Coffee,85 Dean St,9.1,766,40.687295,-73.989907,NaN,0.665495
2,East One Coffee Roasters,384 Court St,9.1,287,40.681128,-73.996526,https://www.seamless.com/menu/east-one-coffee-...,1.164634
3,Joe Coffee,102 Hicks St,8.7,114,40.698673,-73.994358,NaN,0.792574
4,White Noise Coffee Co.,71 Smith St,8.2,68,40.689736,-73.988203,NaN,0.642694
5,Bluestone Lane,55 Prospect St,8.0,269,40.700880,-73.988060,https://www.seamless.com/menu/bluestone-lane-c...,1.205066
6,Cobble Hill Coffee Shop,314 Court St,8.0,125,40.683434,-73.995426,https://www.seamless.com/menu/cobble-hill-coff...,0.904777
7,Gregory's Coffee,16 Court St,7.7,135,40.693560,-73.990670,https://www.grubhub.com/restaurant/gregorys-co...,0.456915
8,D'Amico Coffee Roasters,309 Court St,7.6,120,40.683593,-73.995307,NaN,0.887113
9,Absolute Coffee,327 Atlantic Ave,7.3,216,40.688079,-73.987693,NaN,0.759357


In [23]:
show_grid(foursquare_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [21]:
markers = [(row.lat, row.lng) for row in foursquare_df.itertuples()]
marker_hover = ["%s: %s (%s)" % (row.name, row.rating, row.nratings) for row in foursquare_df.itertuples()]

info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Foursquare rating</dt><dd>{rating}</dd>
<dt>Foursquare reviews</dt><dd>{nratings}</dd>
</dl>
"""
marker_info = [info_box_template.format(**d_item) for i, d_item in foursquare_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=location_coords, zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [22]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map

## Deduplicate and merge

In [34]:
# put everything in one big df
pd.set_option('display.max_rows', None)

gmaps_df_copy = gmaps_df.copy()
gmaps_df_copy['source'] = '0_gmaps'

yelp_df_copy = yelp_df.copy() 
yelp_df_copy['source'] = '1_yelp'

foursquare_df_copy = foursquare_df.copy()
foursquare_df_copy['source'] = '2_foursquare'

venues_df = pd.concat([gmaps_df_copy, yelp_df_copy, foursquare_df_copy]).reset_index()
venues_df['latlong'] = venues_df[['lat','lng']].apply(tuple, axis=1)

venues_df.sort_values('name')

,index,name,address,rating,nratings,lat,lng,distance,source,url,latlong
3,3,About coffee,187 State St,4.7,52,40.690322,-73.991653,0.346982,0_gmaps,NaN,"(40.6903215, -73.99165289999999)"
30,1,Absolute Coffee,327 Atlantic Ave,4.5,177,40.688270,-73.987569,0.757914,1_yelp,https://www.yelp.com/biz/absolute-coffee-brook...,"(40.68827, -73.987569)"
9,9,Absolute Coffee,327 Atlantic Ave,4.4,268,40.688194,-73.987637,0.757055,0_gmaps,NaN,"(40.6881938, -73.9876371)"
57,9,Absolute Coffee,327 Atlantic Ave,7.3,216,40.688079,-73.987693,0.759357,2_foursquare,NaN,"(40.68807946939255, -73.98769332950604)"
26,26,Au Bon Pain,70 Myrtle Ave,3.8,154,40.693604,-73.986540,0.782624,0_gmaps,NaN,"(40.6936043, -73.9865401)"
29,0,Blossom Ice Cream,196 Court St,4.5,215,40.687279,-73.993973,0.492972,1_yelp,https://www.yelp.com/biz/blossom-ice-cream-bro...,"(40.6872787475586, -73.9939727783203)"
31,2,Blue Bottle Coffee,85 Dean St,4.5,139,40.687360,-73.989770,0.668568,1_yelp,https://www.yelp.com/biz/blue-bottle-coffee-br...,"(40.68736, -73.9897699)"
4,4,Blue Bottle Coffee,85 Dean St,4.6,392,40.687351,-73.989750,0.670440,0_gmaps,NaN,"(40.68735119999999, -73.9897502)"
49,1,Blue Bottle Coffee,85 Dean St,9.1,766,40.687295,-73.989907,0.665495,2_foursquare,NaN,"(40.687295303641385, -73.98990664772262)"
53,5,Bluestone Lane,55 Prospect St,8.0,269,40.700880,-73.988060,1.205066,2_foursquare,https://www.seamless.com/menu/bluestone-lane-c...,"(40.70088000661762, -73.98805975913996)"


In [36]:
# manual fix 
# venues_df.loc[venues_df['address']=='212b Pacific St', 'address'] = '212 Pacific St'
# venues_df

,index,name,address,rating,nratings,lat,lng,distance,source,url,latlong
0,0,Stumptown Coffee Roasters,212 Pacific St,4.7,190,40.688831,-73.992174,0.410008,0_gmaps,NaN,"(40.6888305, -73.9921739)"
1,1,D'Amico Coffee,309 Court St,4.7,130,40.683479,-73.995175,0.899972,0_gmaps,NaN,"(40.6834788, -73.9951753)"
2,2,Kaigo Coffee Room,139 Bridge Park Dr,4.7,81,40.693491,-74.000994,0.517465,0_gmaps,NaN,"(40.6934911, -74.0009937)"
3,3,About coffee,187 State St,4.7,52,40.690322,-73.991653,0.346982,0_gmaps,NaN,"(40.6903215, -73.99165289999999)"
4,4,Blue Bottle Coffee,85 Dean St,4.6,392,40.687351,-73.989750,0.670440,0_gmaps,NaN,"(40.68735119999999, -73.9897502)"
5,5,East One Coffee Roasters,384 Court St,4.5,348,40.681188,-73.996620,1.158717,0_gmaps,NaN,"(40.6811876, -73.9966204)"
6,6,The Little Sweet Café,77 Hoyt St,4.5,240,40.688093,-73.986608,0.838769,0_gmaps,NaN,"(40.6880933, -73.9866079)"
7,7,WHITE NOISE COFFEE CO.,57 Smith St,4.5,71,40.689698,-73.988224,0.642386,0_gmaps,NaN,"(40.6896983, -73.9882236)"
8,8,Devoción,276 Livingston St,4.4,377,40.688557,-73.983459,1.064517,0_gmaps,NaN,"(40.6885567, -73.98345909999999)"
9,9,Absolute Coffee,327 Atlantic Ave,4.4,268,40.688194,-73.987637,0.757055,0_gmaps,NaN,"(40.6881938, -73.9876371)"


In [37]:
# run dedupe algorithm using name, address as default texts, latlong as latlong
venues_df2 = pandas_dedupe.dedupe_dataframe(venues_df, ['name', 'address', ('latlong', 'LatLong')])


Importing data ...
Reading from dedupe_dataframe_learned_settings
Clustering...
# duplicate sets 39


In [38]:
# view clustering
venues_df['cluster'] = venues_df2['cluster id']
venues_df = venues_df.sort_values(['cluster', 'source'])[['cluster', 'name', 'address', 'rating', 'nratings', 'lat', 'lng', 'source']]
venues_df

,cluster,name,address,rating,nratings,lat,lng,source
0,0,Stumptown Coffee Roasters,212 Pacific St,4.7,190,40.6888305,-73.9921739,0_gmaps
48,0,Stumptown Coffee Roasters,212 Pacific St,9.2,207,40.68894174493841,-73.99214088428779,2_foursquare
1,1,D'Amico Coffee,309 Court St,4.7,130,40.6834788,-73.9951753,0_gmaps
56,1,D'Amico Coffee Roasters,309 Court St,7.6,120,40.683593021952916,-73.99530657474676,2_foursquare
2,2,Kaigo Coffee Room,139 Bridge Park Dr,4.7,81,40.6934911,-74.0009937,0_gmaps
34,2,Kaigo Coffee Room,139 Bridge Park Dr,4.5,54,40.6936075465661,-74.0010171957826,1_yelp
4,3,Blue Bottle Coffee,85 Dean St,4.6,392,40.68735119999999,-73.9897502,0_gmaps
31,3,Blue Bottle Coffee,85 Dean St,4.5,139,40.68736,-73.9897699,1_yelp
49,3,Blue Bottle Coffee,85 Dean St,9.1,766,40.687295303641385,-73.98990664772262,2_foursquare
5,4,East One Coffee Roasters,384 Court St,4.5,348,40.6811876,-73.9966204,0_gmaps


In [39]:
# group by clusters
cluster_df = venues_df.groupby('cluster')[['name', 'address', 'lat', 'lng', 'source']] \
    .first() \
    .reset_index()
cluster_df

,cluster,name,address,lat,lng,source
0,0,Stumptown Coffee Roasters,212 Pacific St,40.6888305,-73.9921739,0_gmaps
1,1,D'Amico Coffee,309 Court St,40.6834788,-73.9951753,0_gmaps
2,2,Kaigo Coffee Room,139 Bridge Park Dr,40.6934911,-74.0009937,0_gmaps
3,3,Blue Bottle Coffee,85 Dean St,40.68735119999999,-73.9897502,0_gmaps
4,4,East One Coffee Roasters,384 Court St,40.6811876,-73.9966204,0_gmaps
5,5,WHITE NOISE COFFEE CO.,57 Smith St,40.6896983,-73.9882236,0_gmaps
6,6,Absolute Coffee,327 Atlantic Ave,40.6881938,-73.9876371,0_gmaps
7,7,Joe Coffee Company,102 Hicks St,40.6986734,-73.9943579,0_gmaps
8,8,Gregorys Coffee,16 Court St,40.6936215,-73.99085989999999,0_gmaps
9,9,Bluestone Lane DUMBO Café,55 Prospect St,40.7008302,-73.9883345,0_gmaps


In [40]:
# make markers on clusters
# add ratings on all rows

markers = [(float(row.lat), float(row.lng)) for row in cluster_df.itertuples()]
marker_hover = ["%s" % (row.name) for row in cluster_df.itertuples()]

# make a dict by cluster, initialize rating string to ''
marker_dict = {i: {'name': row['name'],
                   'address': row['address'],
                   'lat': row['lat'],
                   'lng': row['lng'],
                   'rate_str': '',
                  } for i, row in cluster_df.iterrows()}

# add all rating strings
for i, row in venues_df.iterrows():
    cluster = row['cluster']
    if row.source[2:] == 'gmaps':
        marker_dict[cluster]['rate_str'] += "<dt>Google rating</dt><dd>%s (%s reviews)</dd>\n" % (row.rating, row.nratings)
    elif row.source[2:] == 'yelp':
        marker_dict[cluster]['rate_str'] += "<dt>Yelp rating</dt><dd>%s (%s reviews)</dd>\n" % (row.rating, row.nratings)
    elif row.source[2:] == 'foursquare':
        marker_dict[cluster]['rate_str'] += "<dt>Foursquare rating</dt><dd>%s (%s reviews)</dd>\n" % (row.rating, row.nratings)
    
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
{rate_str}

</dl>
"""    

marker_info = [info_box_template.format(**d_item) for i, d_item in marker_dict.items()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=location_coords, zoom_level=14)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [41]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map


In [42]:
merge_df = cluster_df \
    .merge(venues_df.loc[venues_df['source']=='0_gmaps'][['cluster','rating']], on='cluster', how='outer') \
    .rename(columns={'rating': 'gmaps_rating'})
merge_df['gmaps_rating_std'] = StandardScaler().fit_transform(merge_df[['gmaps_rating']])

merge_df = merge_df \
    .merge(venues_df.loc[venues_df['source']=='1_yelp'][['cluster','rating']], on='cluster', how='outer') \
    .rename(columns={'rating': 'yelp_rating'})
merge_df['yelp_rating_std'] = StandardScaler().fit_transform(merge_df[['yelp_rating']])

merge_df = merge_df \
    .merge(venues_df.loc[venues_df['source']=='2_foursquare'][['cluster','rating']], on='cluster', how='outer') \
    .rename(columns={'rating': 'foursquare_rating'})
merge_df['foursquare_rating_std'] = StandardScaler().fit_transform(merge_df[['foursquare_rating']])

merge_df 

,cluster,name,address,lat,lng,source,gmaps_rating,gmaps_rating_std,yelp_rating,yelp_rating_std,foursquare_rating,foursquare_rating_std
0,0,Stumptown Coffee Roasters,212 Pacific St,40.6888305,-73.9921739,0_gmaps,4.7,1.444623,NaN,NaN,9.2,1.467708
1,1,D'Amico Coffee,309 Court St,40.6834788,-73.9951753,0_gmaps,4.7,1.444623,NaN,NaN,7.6,-0.050611
2,2,Kaigo Coffee Room,139 Bridge Park Dr,40.6934911,-74.0009937,0_gmaps,4.7,1.444623,4.5,1.248075,NaN,NaN
3,3,Blue Bottle Coffee,85 Dean St,40.68735119999999,-73.9897502,0_gmaps,4.6,1.122361,4.5,1.248075,9.1,1.372813
4,4,East One Coffee Roasters,384 Court St,40.6811876,-73.9966204,0_gmaps,4.5,0.800099,4.0,-0.069338,9.1,1.372813
5,5,WHITE NOISE COFFEE CO.,57 Smith St,40.6896983,-73.9882236,0_gmaps,4.5,0.800099,4.5,1.248075,NaN,NaN
6,6,Absolute Coffee,327 Atlantic Ave,40.6881938,-73.9876371,0_gmaps,4.4,0.477837,4.5,1.248075,7.3,-0.335295
7,7,Joe Coffee Company,102 Hicks St,40.6986734,-73.9943579,0_gmaps,4.4,0.477837,3.5,-1.386750,8.7,0.993233
8,8,Gregorys Coffee,16 Court St,40.6936215,-73.99085989999999,0_gmaps,4.4,0.477837,4.0,-0.069338,7.7,0.044284
9,9,Bluestone Lane DUMBO Café,55 Prospect St,40.7008302,-73.9883345,0_gmaps,4.3,0.155575,4.0,-0.069338,8.0,0.328969


In [43]:
merge_df['distance'] = merge_df.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
                                      axis=1)
merge_df


,cluster,name,address,lat,lng,source,gmaps_rating,gmaps_rating_std,yelp_rating,yelp_rating_std,foursquare_rating,foursquare_rating_std,distance
0,0,Stumptown Coffee Roasters,212 Pacific St,40.6888305,-73.9921739,0_gmaps,4.7,1.444623,NaN,NaN,9.2,1.467708,0.410008
1,1,D'Amico Coffee,309 Court St,40.6834788,-73.9951753,0_gmaps,4.7,1.444623,NaN,NaN,7.6,-0.050611,0.899972
2,2,Kaigo Coffee Room,139 Bridge Park Dr,40.6934911,-74.0009937,0_gmaps,4.7,1.444623,4.5,1.248075,NaN,NaN,0.517465
3,3,Blue Bottle Coffee,85 Dean St,40.68735119999999,-73.9897502,0_gmaps,4.6,1.122361,4.5,1.248075,9.1,1.372813,0.670440
4,4,East One Coffee Roasters,384 Court St,40.6811876,-73.9966204,0_gmaps,4.5,0.800099,4.0,-0.069338,9.1,1.372813,1.158717
5,5,WHITE NOISE COFFEE CO.,57 Smith St,40.6896983,-73.9882236,0_gmaps,4.5,0.800099,4.5,1.248075,NaN,NaN,0.642386
6,6,Absolute Coffee,327 Atlantic Ave,40.6881938,-73.9876371,0_gmaps,4.4,0.477837,4.5,1.248075,7.3,-0.335295,0.757055
7,7,Joe Coffee Company,102 Hicks St,40.6986734,-73.9943579,0_gmaps,4.4,0.477837,3.5,-1.386750,8.7,0.993233,0.792574
8,8,Gregorys Coffee,16 Court St,40.6936215,-73.99085989999999,0_gmaps,4.4,0.477837,4.0,-0.069338,7.7,0.044284,0.446337
9,9,Bluestone Lane DUMBO Café,55 Prospect St,40.7008302,-73.9883345,0_gmaps,4.3,0.155575,4.0,-0.069338,8.0,0.328969,1.188479


In [44]:
# simple average score
merge_df['meanrating'] = np.nanmean(merge_df[['gmaps_rating_std', 'yelp_rating_std', 'foursquare_rating_std']], axis=1)
merge_df.sort_values('meanrating', ascending=False)[['name', 'address', 'gmaps_rating', 'yelp_rating', 'foursquare_rating', 'meanrating']]


,name,address,gmaps_rating,yelp_rating,foursquare_rating,meanrating
0,Stumptown Coffee Roasters,212 Pacific St,4.7,NaN,9.2,1.456165
17,About coffee,187 State St,4.7,NaN,NaN,1.444623
2,Kaigo Coffee Room,139 Bridge Park Dr,4.7,4.5,NaN,1.346349
30,The Little Sweet Cafe,77B Hoyt St,NaN,4.5,NaN,1.248075
29,Blossom Ice Cream,196 Court St,NaN,4.5,NaN,1.248075
3,Blue Bottle Coffee,85 Dean St,4.6,4.5,9.1,1.247750
5,WHITE NOISE COFFEE CO.,57 Smith St,4.5,4.5,NaN,1.024087
18,The Little Sweet Café,77 Hoyt St,4.5,NaN,NaN,0.800099
4,East One Coffee Roasters,384 Court St,4.5,4.0,9.1,0.701191
1,D'Amico Coffee,309 Court St,4.7,NaN,7.6,0.697006


In [50]:
# bayes score
rating_cols = ['gmaps_rating_std', 'yelp_rating_std', 'foursquare_rating_std']
merge_df['nratings'] = merge_df[rating_cols].count(axis=1)
nratings_mean = np.mean(merge_df['nratings'])
rating_avg = np.nanmean(merge_df[rating_cols])
merge_df['w'] = merge_df['nratings']/(merge_df['nratings'] + nratings_mean)
merge_df['R'] = np.mean(merge_df[rating_cols], axis=1)
print('mean number of ratings', nratings_mean)
print('average rating', rating_avg)
merge_df['bayes_score'] = merge_df['w'] * merge_df['R'] + (1 - merge_df['w']) * rating_avg

show_grid(merge_df.sort_values('bayes_score', ascending=False)[['name', 'address', 'distance', 'gmaps_rating', 'yelp_rating', 'foursquare_rating', 'nratings', 'bayes_score']])

mean number of ratings 1.6153846153846154
average rating 8.353106566227368e-16


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [46]:
def dedupe(dedupe_list):

    for i, source_df in enumerate(dedupe_list):
        source_df['source'] = i
    venues_df = pd.concat(dedupe_list).reset_index()
    venues_df['latlong'] = venues_df[['lat','lng']].apply(tuple, axis=1)
    venues_df['shortname'] = venues_df['name'].apply(lambda n: n[:25])

    # dedupe and assign cluster id
    venues_df2 = pandas_dedupe.dedupe_dataframe(venues_df, ['shortname', 'address', ('latlong', 'LatLong')])
    venues_df['cluster'] = venues_df2['cluster id']
    venues_df = venues_df.sort_values(['cluster', 'source'])[['cluster', 'name', 'address', 'rating', 'nratings', 'lat', 'lng', 'source']]

    # group by clusters, uniquify name
    cluster_df = venues_df.groupby('cluster')[['name', 'address', 'lat', 'lng', 'source']] \
                          .first() \
                          .reset_index()

    # merge ratings by source
    merge_df = cluster_df \
        .merge(venues_df.loc[venues_df['source']=='0'][['cluster','rating']], on='cluster', how='outer') \
        .rename(columns={'rating': 'gmaps_rating'})
    merge_df['gmaps_rating_std'] = StandardScaler().fit_transform(merge_df[['gmaps_rating']])

    merge_df = merge_df \
        .merge(venues_df.loc[venues_df['source']=='1'][['cluster','rating']], on='cluster', how='outer') \
        .rename(columns={'rating': 'yelp_rating'})
    merge_df['yelp_rating_std'] = StandardScaler().fit_transform(merge_df[['yelp_rating']])

    merge_df = merge_df \
        .merge(venues_df.loc[venues_df['source']=='2'][['cluster','rating']], on='cluster', how='outer') \
        .rename(columns={'rating': 'foursquare_rating'})
    merge_df['foursquare_rating_std'] = StandardScaler().fit_transform(merge_df[['foursquare_rating']])

    # bayes score
    rating_cols = ['gmaps_rating_std', 'yelp_rating_std', 'foursquare_rating_std']
    merge_df['nratings'] = merge_df[rating_cols].count(axis=1)
    nratings_mean = np.mean(merge_df['nratings'])
    rating_avg = np.nanmean(merge_df[rating_cols])
    merge_df['w'] = merge_df['nratings']/(merge_df['nratings'] + nratings_mean)
    merge_df['R'] = np.mean(merge_df[rating_cols], axis=1)
    merge_df['bayes_score'] = merge_df['w'] * merge_df['R'] + (1 - merge_df['w']) * rating_avg
    merge_df = merge_df.sort_values('bayes_score', ascending=False)
    merge_df[['name', 'address', 'gmaps_rating', 'yelp_rating', 'foursquare_rating', 'nratings', 'bayes_score']]
    return merge_df

    
dedupe_list = list(filter(lambda df: df is not None, [gmaps_df, yelp_df, foursquare_df]))

dedupe(dedupe_list)

Importing data ...
Reading from dedupe_dataframe_learned_settings
Clustering...
# duplicate sets 40


,cluster,name,address,lat,lng,source,gmaps_rating,gmaps_rating_std,yelp_rating,yelp_rating_std,foursquare_rating,foursquare_rating_std,nratings,w,R,bayes_score
2,2,Blue Bottle Coffee,85 Dean St,40.68735119999999,-73.9897502,0,4.6,1.122361,4.5,1.248075,9.1,1.372813,3,0.655738,1.247750,0.818197
1,1,Kaigo Coffee Room,139 Bridge Park Dr,40.6934911,-74.0009937,0,4.7,1.444623,4.5,1.248075,NaN,NaN,2,0.559441,1.346349,0.753202
4,4,WHITE NOISE COFFEE CO.,57 Smith St,40.6896983,-73.9882236,0,4.5,0.800099,4.5,1.248075,NaN,NaN,2,0.559441,1.024087,0.572916
36,36,Stumptown Coffee Roasters,212 Pacific St,40.68894174493841,-73.99214088428779,2,NaN,NaN,NaN,NaN,9.2,1.467708,1,0.388350,1.467708,0.569984
16,16,Stumptown Coffee Roasters,212b Pacific St,40.6888305,-73.9921739,0,4.7,1.444623,NaN,NaN,NaN,NaN,1,0.388350,1.444623,0.561019
17,17,About coffee,187 State St,40.6903215,-73.99165289999999,0,4.7,1.444623,NaN,NaN,NaN,NaN,1,0.388350,1.444623,0.561019
30,30,The Little Sweet Cafe,77B Hoyt St,40.6881418,-73.9867324,1,NaN,NaN,4.5,1.248075,NaN,NaN,1,0.388350,1.248075,0.484689
29,29,Blossom Ice Cream,196 Court St,40.6872787475586,-73.9939727783203,1,NaN,NaN,4.5,1.248075,NaN,NaN,1,0.388350,1.248075,0.484689
3,3,East One Coffee Roasters,384 Court St,40.6811876,-73.9966204,0,4.5,0.800099,4.0,-0.069338,9.1,1.372813,3,0.655738,0.701191,0.459798
0,0,D'Amico Coffee,309 Court St,40.6834788,-73.9951753,0,4.7,1.444623,NaN,NaN,7.6,-0.050611,2,0.559441,0.697006,0.389933


In [ ]:
# app.py

location = '40.6915812,-73.9954095'
keyword = 'pizza'
ltype = 'establishment'
rankby = 'distance'

gmaps_df = gmaps_get_df(location, keyword)
yelp_df = yelp_get_df(location, keyword)
foursquare_df = foursquare_get_df(location, keyword)
dedupe_list = list(filter(lambda df: df is not None, [gmaps_df, yelp_df, foursquare_df]))
print(len(gmaps_df), len(yelp_df), len(foursquare_df))
print("Deduping %d dataframes" % (len(list(dedupe_list))))
dedupe(dedupe_list).to_json()
